## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-03-21-51-16 +0000,bbc,Hamas says it agrees to release Israeli hostag...,https://www.bbc.com/news/articles/cdxq7zp7002o...
1,2025-10-03-21-50-53 +0000,nypost,Bitcoin nears record high in latest sign it ma...,https://nypost.com/2025/10/03/business/bitcoin...
2,2025-10-03-21-50-41 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/10/03/world/...
3,2025-10-03-21-49-16 +0000,cbc,Federal budget watchdog's comments are 'just w...,https://www.cbc.ca/news/politics/parliamentary...
4,2025-10-03-21-49-04 +0000,nyt,Supreme Court Lets Trump Revoke Deportation Pr...,https://www.nytimes.com/2025/10/03/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2367/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
31,trump,54
109,shutdown,17
113,attack,16
120,will,16
115,manchester,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
61,2025-10-03-20-03-20 +0000,nypost,Hamas agrees to release all Israeli hostages p...,https://nypost.com/2025/10/03/world-news/hamas...,125
59,2025-10-03-20-27-00 +0000,wsj,The Trump administration has frozen $2.1 billi...,https://www.wsj.com/politics/policy/trump-admi...,118
71,2025-10-03-19-37-00 +0000,wsj,Trump ordered another lethal military strike o...,https://www.wsj.com/politics/national-security...,107
290,2025-10-03-01-13-09 +0000,nyt,Trump Escalates Attacks on Democrats in a Gove...,https://www.nytimes.com/2025/10/02/us/politics...,102
28,2025-10-03-21-16-59 +0000,wapo,Supreme Court will let Trump end protected sta...,https://www.washingtonpost.com/politics/2025/1...,101


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
61,125,2025-10-03-20-03-20 +0000,nypost,Hamas agrees to release all Israeli hostages p...,https://nypost.com/2025/10/03/world-news/hamas...
200,80,2025-10-03-12-00-08 +0000,wapo,One victim in Yom Kippur synagogue attack in E...,https://www.washingtonpost.com/world/2025/10/0...
293,77,2025-10-03-01-00-00 +0000,wsj,"Inside the White House, concerns are growing t...",https://www.wsj.com/politics/policy/trump-gove...
12,55,2025-10-03-21-38-40 +0000,nypost,Sean ‘Diddy’ Combs still faces nearly 70 lawsu...,https://nypost.com/2025/10/03/us-news/sean-did...
71,53,2025-10-03-19-37-00 +0000,wsj,Trump ordered another lethal military strike o...,https://www.wsj.com/politics/national-security...
166,49,2025-10-03-14-23-50 +0000,nypost,Cops probe whether Manchester synagogue terror...,https://nypost.com/2025/10/03/world-news/manch...
162,42,2025-10-03-14-42-11 +0000,latimes,Supreme Court will decide if gun owners may ca...,https://www.latimes.com/politics/story/2025-10...
98,40,2025-10-03-18-16-44 +0000,nypost,Lorde off Apple Music in Israel after yelling ...,https://nypost.com/2025/10/03/business/lorde-o...
210,39,2025-10-03-11-01-00 +0000,startribune,Unemployed Minnesotans face longer job searche...,https://www.startribune.com/long-term-unemploy...
215,37,2025-10-03-10-46-15 +0000,nypost,Sarah Mullally named the first female Archbish...,https://nypost.com/2025/10/03/world-news/churc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
